# Day6_0: pandas 고급 조작 (groupby, pivot, merge)

## 📚 학습 목표

**Part 1: 기초**
1. groupby로 기본 집계 수행하기 (sum, mean, count)
2. 다중 컬럼 groupby로 세분화된 집계하기
3. agg() 함수로 복합 집계 실행하기
4. transform() vs apply() 차이 이해하기
5. Plotly로 집계 결과 시각화하기

**Part 2: 심화**
1. pivot_table()로 데이터 요약하기
2. merge()로 데이터 조인하기 (inner, outer, left, right)
3. concat()으로 데이터 결합하기
4. px.imshow()로 피벗 테이블 히트맵 그리기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 엑셀 대응 | pandas 장점 |
|------|----------|-------------|
| groupby | 피벗 테이블 | 프로그래밍 가능한 집계, 복잡한 조건 |
| pivot_table | 피벗 테이블 | 자동 집계, 다차원 분석 |
| merge | VLOOKUP | 다중 키 조인, 자동 매칭 |
| concat | 복사+붙여넣기 | 자동 정렬, 인덱스 관리 |

**분석가 관점**: 엑셀에서 수작업으로 하던 피벗, VLOOKUP, 데이터 통합을 pandas에서는 코드 몇 줄로 자동화할 수 있습니다!

---

# Part 1: 기초

---

## 1.1 groupby 기본 집계

### groupby란?

- **Split-Apply-Combine** 패턴
- 그룹별로 데이터를 나누고(Split), 함수를 적용하고(Apply), 결과를 합칩니다(Combine)
- 엑셀의 피벗 테이블과 유사하지만 더 강력합니다

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 샘플 매출 데이터 생성
sales_data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산', '서울', '대구', '대구', '부산'],
    'product': ['노트북', '마우스', '노트북', '키보드', '노트북', '마우스', '키보드', '마우스'],
    'sales': [1200, 30, 1000, 80, 1500, 25, 75, 35],
    'quantity': [2, 5, 1, 3, 3, 4, 2, 6]
})

print("원본 데이터:")
print(sales_data)

원본 데이터:
  region product  sales  quantity
0     서울     노트북   1200         2
1     서울     마우스     30         5
2     부산     노트북   1000         1
3     부산     키보드     80         3
4     서울     노트북   1500         3
5     대구     마우스     25         4
6     대구     키보드     75         2
7     부산     마우스     35         6


### 기본 집계 함수

In [2]:
# 1. 지역별 총 매출     #⭐코딩테스트나 BA들이 밥먹듯이 하는거니깐 잘해야함⭐
region_sales = sales_data.groupby(['region'])['sales'].sum()  #region중심으로 집계하겠다! 
print("지역별 총 매출:")         #집계기준대상  #집계할 컬럼               #만약 여러 컬럼 기준이면 리스트로 넣으면 됨
print(region_sales)
print()

# 2. 지역별 총 매출 시각화
fig = px.bar(region_sales, x=region_sales.index, y=region_sales.values, title='지역별 총 매출')
fig.show()

지역별 총 매출:
region
대구     100
부산    1115
서울    2730
Name: sales, dtype: int64



In [ ]:
# 2. 지역별 평균 매출
region_avg = sales_data.groupby('region')['sales'].mean()
print("지역별 평균 매출:")
print(region_avg)
print()

In [ ]:
# 3. 지역별 거래 건수
region_count = sales_data.groupby('region')['sales'].count()
print("지역별 거래 건수:")
print(region_count)

#열이 한 개인 시리즈 형태에지만   reset.index()를 쓰면 인덱스에 있던 것이 컬럼으로 변함. 

지역별 거래 건수:
region
대구    2
부산    3
서울    3
Name: sales, dtype: int64


### 💡 실무 예시: 지역별 매출 분석

전국 매장의 일일 매출 데이터에서 어느 지역이 가장 많이 팔리는지 빠르게 파악할 수 있습니다.

In [ ]:
# 지역별 총 매출을 DataFrame으로 변환
region_summary = sales_data.groupby('region')['sales'].sum().reset_index()
region_summary.columns = ['지역', '총매출']

print(region_summary)

---

## 1.2 다중 컬럼 groupby

### 여러 기준으로 그룹화하기

In [ ]:
# 지역 + 제품별 총 매출
region_product_sales = sales_data.groupby(['region', 'product'])['sales'].sum()   # <== 컬럼 두개로 groupby할 땐 [ [] ] 이렇게 넣어야함! 
print("지역 + 제품별 총 매출:")
print(region_product_sales)
print()

##지금 이건 복합 인덱스인 상태임. region이랑 product가 인덱스
# reset_index()를 쓰면 데이터프레임으로 옮겨짐

지역 + 제품별 총 매출:
region  product
대구      마우스          25
        키보드          75
부산      노트북        1000
        마우스          35
        키보드          80
서울      노트북        2700
        마우스          30
Name: sales, dtype: int64



In [ ]:

# DataFrame으로 변환
region_product_df = region_product_sales.reset_index()
print("\nDataFrame 형태:")
print(region_product_df)

### 여러 컬럼 동시 집계

In [ ]:
# 지역별 매출 합계와 수량 합계 동시에
region_multi = sales_data.groupby('region')[['sales', 'quantity']].sum()
print("지역별 매출 + 수량 합계:")
print(region_multi)

### 💡 실무 예시: 제품 카테고리별 세분화 분석

"서울에서 노트북이 얼마나 팔렸는가?"처럼 세부 질문에 답할 수 있습니다.

In [ ]:
# 지역 + 제품별 상세 통계  ⭐⭐⭐
detailed_stats = sales_data.groupby(['region', 'product']).agg({    #aggregation집계하다 
    'sales': 'sum',                                                #더 복잡한 집계를 할 때 많이 씀
    'quantity': 'sum'                                           #agg {딕셔너리로 표현}, 각 컬럼마다 다른 집계도 가능! => 실무에선 다 agg씀
}).reset_index()

detailed_stats.columns = ['지역', '제품', '총매출', '총수량']
print(detailed_stats)

   지역   제품   총매출  총수량
0  대구  마우스    25    4
1  대구  키보드    75    2
2  부산  노트북  1000    1
3  부산  마우스    35    6
4  부산  키보드    80    3
5  서울  노트북  2700    5
6  서울  마우스    30    5


---

## 1.3 agg() 함수로 복합 집계

### 여러 집계 함수 동시 적용

In [ ]:
# 지역별 매출 통계 (합계, 평균, 최대, 최소)
region_stats = sales_data.groupby('region')['sales'].agg(['sum', 'mean', 'max', 'min', 'count'])  #얘는 딕셔너리가 아니고 대괄호? 
print("지역별 매출 통계:")                              
print(region_stats)
print()

# 컬럼명 한글로 변경
region_stats.columns = ['합계', '평균', '최대', '최소', '건수']
print("\n컬럼명 변경:")
print(region_stats)

지역별 매출 통계:
         sum        mean   max  min  count
region                                    
대구       100   50.000000    75   25      2
부산      1115  371.666667  1000   35      3
서울      2730  910.000000  1500   30      3


컬럼명 변경:
          합계          평균    최대  최소  건수
region                                
대구       100   50.000000    75  25   2
부산      1115  371.666667  1000  35   3
서울      2730  910.000000  1500  30   3


### 컬럼별 다른 집계 함수 적용

In [ ]:
# 매출은 합계, 수량은 평균
custom_agg = sales_data.groupby('region').agg({
    'sales': 'sum',
    'quantity': 'mean'
})

print("지역별 총매출 + 평균수량:")
print(custom_agg)

### 여러 집계 함수를 리스트로 지정

In [ ]:
# 매출: 합계 + 평균, 수량: 합계만
multi_agg = sales_data.groupby('region').agg({
    'sales': ['sum', 'mean'],       #컬럼별로 집계도 달리하고 싶은데 그 집계할 종류도 여러개 보고 싶을 때! 
    'quantity': 'sum'
})
print("복합 집계:")
print(multi_agg)

In [ ]:
⭐강사님이 추천하시는 방법 (판다스식 표현)⭐

sales_data.groupby(['region']).agg(
    total_sales = ('sales', 'sum') 첫번재는 대상이 되는 컬럼, 두번재는 집계 방법
    z_score_sales=('sales', lamda x: (x-x.mean())/x.std().mean())        # ==> 좀 더 복잡한 집계를 하고 싶다면? 이 정도면 충분! 마스터 한거임.
)

### 💡 실무 예시: 월별 매출 리포트

월말에 보고서를 만들 때 합계, 평균, 최대값을 한 번에 계산할 수 있습니다.

In [ ]:
⭐아래꺼 강사님 버전⭐



In [ ]:
# 제품별 판매 성과 리포트
product_report = sales_data.groupby('product').agg({
    'sales': ['sum', 'mean', 'max'],
    'quantity': ['sum', 'mean']
}).round(1)

print("제품별 판매 성과:")
print(product_report)

---

## 1.4 transform() vs apply()

### transform(): 원본과 같은 크기 반환

In [ ]:
# 지역별 평균 매출을 각 행에 추가
sales_data['region_avg'] = sales_data.groupby('region')['sales'].transform('mean')

print("지역별 평균이 추가된 데이터:")
print(sales_data[['region', 'sales', 'region_avg']])

In [ ]:
# 지역 평균 대비 비율 계산
sales_data['vs_avg_ratio'] = (sales_data['sales'] / sales_data['region_avg'] * 100).round(1)

print("\n지역 평균 대비 비율 (%):")
print(sales_data[['region', 'sales', 'region_avg', 'vs_avg_ratio']])

### apply(): 사용자 정의 함수 적용

In [ ]:
⭐강사님 버전?
sales_data.groupby(['region']).agg(
    region_range = ('sales', lambda x : x.max() - x.min())
)

#아래 거랑 같은 결과임! 다르게 구현한 것

In [ ]:
# 지역별 최대-최소 차이 계산
def sales_range(series):
    return series.max() - series.min()

region_range = sales_data.groupby('region')['sales'].apply(sales_range) #내가 만든 함수를 쓸 수 있음
print("지역별 매출 범위 (최대-최소):")                                     #apply를 써서 구현할 수도 있다는 걸 알아놓기! 
print(region_range)

### 💡 실무 예시: 이상치 탐지

각 그룹의 평균과 비교하여 특이하게 높거나 낮은 값을 찾을 수 있습니다.

In [ ]:
# 평균의 150% 이상인 매출 찾기
high_sales = sales_data[sales_data['vs_avg_ratio'] >= 150]
print("평균보다 50% 이상 높은 매출:")
print(high_sales[['region', 'product', 'sales', 'vs_avg_ratio']])

---

## 1.5 Plotly로 집계 결과 시각화

### px.bar()로 막대 그래프

In [ ]:
#판다스로 바로 plotly 사용하고 싶다면?
#pd.options.plotting.backend = "plotly"    
#판다스 실행하기 전에 이거 한 번 실행하고 하면 됨

In [ ]:
# 지역별 총 매출 막대 그래프
region_total = sales_data.groupby('region')['sales'].sum().reset_index()

fig = px.bar(
    region_total,
    x='region',
    y='sales',
    title='지역별 총 매출',
    labels={'region': '지역', 'sales': '매출 (만원)'},
    text='sales'
)

fig.update_traces(texttemplate='%{text:,}', textposition='outside')    
fig.show()

### px.line()으로 트렌드 시각화

In [ ]:
# 시간별 데이터 생성 (예시)             
time_sales = pd.DataFrame({
    'month': ['1월', '2월', '3월', '4월', '5월'],
    'sales': [1200, 1350, 1100, 1500, 1650]
})

fig = px.line(              #추세선 그려주는 거임
    time_sales,
    x='month',
    y='sales',
    title='월별 매출 트렌드',
    labels={'month': '월', 'sales': '매출 (만원)'},
    markers=True
)

fig.show()

### 그룹별 비교 막대 그래프

In [ ]:
# 지역 + 제품별 매출
region_product = sales_data.groupby(['region', 'product'])['sales'].sum().reset_index()

fig = px.bar(
    region_product,
    x='region',
    y='sales',
    color='product',
    title='지역별 제품 매출 비교',
    labels={'region': '지역', 'sales': '매출 (만원)', 'product': '제품'},
    barmode='group'
)

fig.show()

### 💡 실무 예시: 대시보드용 집계 차트

경영진에게 보고할 때 groupby + Plotly 조합으로 인터랙티브 차트를 빠르게 만들 수 있습니다.

In [ ]:
# 제품별 총 매출 (정렬)
product_total = sales_data.groupby('product')['sales'].sum().reset_index()  #제품별 매출을 합계하고 있음,
product_total = product_total.sort_values('sales', ascending=True)  #매출기준으로 오름차순으로 정렬

fig = px.bar(
    product_total,
    x='sales',
    y='product',
    orientation='h',
    title='제품별 총 매출 (정렬)',
    labels={'sales': '매출 (만원)', 'product': '제품'},
    text='sales'
)

fig.update_traces(texttemplate='%{text:,}', textposition='outside')
fig.show()

---

# Part 2: 심화

---

## 2.1 pivot_table()로 데이터 요약

### pivot_table이란?

- 엑셀의 피벗 테이블과 동일한 기능
- 행/열/값을 지정하여 2차원 요약 테이블 생성
- 자동으로 집계 함수 적용

In [ ]:
# 지역(행) x 제품(열) 피벗 테이블
pivot_sales = sales_data.pivot_table(       
    values='sales',
    index='region',
    columns='product',
    aggfunc='sum',
    fill_value=0  # 결측치를 0으로 채움
)

print("지역 x 제품 피벗 테이블:")
print(pivot_sales)

### 여러 집계 함수 사용

In [ ]:
# 합계와 평균 동시에
pivot_multi = sales_data.pivot_table(
    values='sales',
    index='region',
    columns='product',
    aggfunc=['sum', 'mean'],
    fill_value=0
)

print("합계 + 평균 피벗:")
print(pivot_multi)

### margins로 총합 추가

In [ ]:
# 행/열 총합 자동 계산
pivot_with_total = sales_data.pivot_table(
    values='sales',
    index='region',
    columns='product',
    aggfunc='sum',
    fill_value=0,
    margins=True,
    margins_name='총합'
)

print("총합 포함 피벗:")
print(pivot_with_total)

### 💡 실무 예시: 월별 제품 매출 요약

"1월에 노트북이 얼마 팔렸고, 2월에는..." 같은 질문에 한눈에 답할 수 있는 요약표를 만듭니다.

In [ ]:
# 월별 제품 매출 데이터 생성
monthly_data = pd.DataFrame({
    'month': ['1월', '1월', '2월', '2월', '3월', '3월'],
    'product': ['노트북', '마우스', '노트북', '마우스', '노트북', '마우스'],
    'sales': [1200, 30, 1350, 35, 1100, 28]
})

monthly_pivot = monthly_data.pivot_table(
    values='sales',
    index='month',
    columns='product',
    aggfunc='sum'
)

print("월별 제품 매출:")
print(monthly_pivot)

---

## 2.2 merge()로 데이터 조인

### merge란?

- SQL의 JOIN과 동일한 개념
- 공통 키(key)를 기준으로 두 DataFrame을 결합
- 조인 방식: inner, outer, left, right

In [ ]:
# 주문 데이터
orders = pd.DataFrame({
    'order_id': [1, 2, 3, 4],
    'product_id': [101, 102, 101, 103],
    'quantity': [2, 5, 3, 1]
})

# 제품 마스터 데이터
products = pd.DataFrame({
    'product_id': [101, 102, 103],
    'product_name': ['노트북', '마우스', '키보드'],
    'price': [1200, 30, 80]
})

print("주문 데이터:")
print(orders)
print()
print("제품 마스터:")
print(products)

### Inner Join (교집합)

In [ ]:
# product_id가 양쪽 모두 있는 것만
merged_inner = pd.merge(orders, products, on='product_id', how='inner')
print("Inner Join 결과:")
print(merged_inner)

### Left Join (왼쪽 기준)

In [ ]:
# orders의 모든 행 유지, products는 매칭되는 것만
merged_left = pd.merge(orders, products, on='product_id', how='left')
print("Left Join 결과:")
print(merged_left)

### Outer Join (합집합)

In [ ]:
# 양쪽 모든 행 유지
products_extended = pd.DataFrame({
    'product_id': [101, 102, 104],  # 104는 주문 없음
    'product_name': ['노트북', '마우스', '모니터'],
    'price': [1200, 30, 350]
})

merged_outer = pd.merge(orders, products_extended, on='product_id', how='outer')
print("Outer Join 결과:")
print(merged_outer)

### 다중 키 조인

In [ ]:
# 지역 + 제품 기준 조인
sales_detail = pd.DataFrame({
    'region': ['서울', '부산', '서울'],
    'product': ['노트북', '노트북', '마우스'],
    'sales': [1200, 1000, 30]
})

target_info = pd.DataFrame({
    'region': ['서울', '부산', '대구'],
    'product': ['노트북', '노트북', '노트북'],
    'target': [1500, 900, 800]
})

merged_multi = pd.merge(sales_detail, target_info, on=['region', 'product'], how='left')    # ON 절에 리스트로 써서 가능!
print("다중 키 조인:")      #suffixes 옵션도 있음
print(merged_multi)

### 💡 실무 예시: 주문 내역에 제품 정보 추가

엑셀의 VLOOKUP처럼 주문 데이터에 제품 이름과 가격을 자동으로 매칭합니다.

In [ ]:
# 주문 금액 계산
merged_inner['total_amount'] = merged_inner['quantity'] * merged_inner['price']
print("주문 상세 내역:")
print(merged_inner[['order_id', 'product_name', 'quantity', 'price', 'total_amount']])

---

## 2.3 concat()으로 데이터 결합

### concat이란?

- 여러 DataFrame을 위아래(axis=0) 또는 좌우(axis=1)로 붙이기
- 엑셀의 복사+붙여넣기와 유사하지만 자동으로 인덱스 관리

In [ ]:
# 1월 매출
jan_sales = pd.DataFrame({
    'product': ['노트북', '마우스'],
    'sales': [1200, 30]
})

# 2월 매출
feb_sales = pd.DataFrame({
    'product': ['노트북', '키보드'],
    'sales': [1350, 85]
})

print("1월 매출:")
print(jan_sales)
print()
print("2월 매출:")
print(feb_sales)

### 위아래로 결합 (axis=0)

In [ ]:
# 두 데이터를 위아래로 합치기
combined = pd.concat([jan_sales, feb_sales], axis=0)
print("위아래 결합:")
print(combined)
print()

# 인덱스 재설정
combined_reset = pd.concat([jan_sales, feb_sales], axis=0, ignore_index=True)
print("인덱스 재설정:")
print(combined_reset)

### 좌우로 결합 (axis=1)

In [ ]:
# 제품 정보
product_names = pd.DataFrame({
    'product': ['노트북', '마우스', '키보드']
})

# 가격 정보
product_prices = pd.DataFrame({
    'price': [1200, 30, 80]
})

# 좌우 결합
combined_cols = pd.concat([product_names, product_prices], axis=1)
print("좌우 결합:")
print(combined_cols)

### keys로 구분 레이블 추가

In [ ]:
# 월별 구분 추가
monthly_combined = pd.concat([jan_sales, feb_sales], keys=['1월', '2월'])
print("월별 구분 레이블:")
print(monthly_combined)

### 💡 실무 예시: 여러 파일 병합

각 지역별로 만들어진 엑셀 파일을 하나의 DataFrame으로 통합할 때 사용합니다.

In [ ]:
# 지역별 데이터 (파일에서 읽었다고 가정)
seoul_data = pd.DataFrame({'region': ['서울', '서울'], 'sales': [1200, 1500]})
busan_data = pd.DataFrame({'region': ['부산', '부산'], 'sales': [1000, 1100]})
daegu_data = pd.DataFrame({'region': ['대구', '대구'], 'sales': [800, 850]})

# 전국 데이터 통합
all_data = pd.concat([seoul_data, busan_data, daegu_data], ignore_index=True)
print("전국 데이터 통합:")
print(all_data)

---

## 2.4 px.imshow()로 피벗 테이블 히트맵

### 히트맵이란?

- 숫자를 색상으로 표현하여 패턴을 시각적으로 파악
- 피벗 테이블의 값을 색으로 구분하여 한눈에 비교

In [ ]:
# 지역 x 제품 피벗 테이블
pivot_for_heatmap = sales_data.pivot_table(     #피벗테이블을 먼저 만듦
    values='sales',
    index='region',
    columns='product',
    aggfunc='sum',
    fill_value=0
)

print("피벗 테이블:")
print(pivot_for_heatmap)

### px.imshow()로 히트맵 그리기

In [ ]:
# 히트맵 생성
fig = px.imshow(
    pivot_for_heatmap,
    labels=dict(x="제품", y="지역", color="매출"),
    x=pivot_for_heatmap.columns,
    y=pivot_for_heatmap.index,
    title="지역 x 제품 매출 히트맵",
    text_auto=True,
    color_continuous_scale='Blues'
)

fig.show()

### 다양한 컬러맵 적용

In [ ]:
# RdYlGn (빨강-노랑-초록) 컬러맵
fig = px.imshow(
    pivot_for_heatmap,
    labels=dict(x="제품", y="지역", color="매출"),
    x=pivot_for_heatmap.columns,
    y=pivot_for_heatmap.index,
    title="지역 x 제품 매출 히트맵 (RdYlGn)",
    text_auto=True,
    color_continuous_scale='RdYlGn'
)

fig.show()

### 💡 실무 예시: 월별 제품 판매 패턴 분석

어느 달에 어느 제품이 잘 팔리는지 색상으로 한눈에 파악할 수 있습니다.

In [ ]:
# 월별 제품 매출 히트맵용 데이터
monthly_product_data = pd.DataFrame({
    'month': ['1월', '1월', '1월', '2월', '2월', '2월', '3월', '3월', '3월'],
    'product': ['노트북', '마우스', '키보드', '노트북', '마우스', '키보드', '노트북', '마우스', '키보드'],
    'sales': [1200, 30, 80, 1350, 35, 85, 1100, 28, 75]
})

monthly_pivot = monthly_product_data.pivot_table(
    values='sales',
    index='month',
    columns='product',
    aggfunc='sum'
)

fig = px.imshow(
    monthly_pivot,
    labels=dict(x="제품", y="월", color="매출"),
    x=monthly_pivot.columns,
    y=monthly_pivot.index,
    title="월별 제품 매출 패턴",
    text_auto=True,
    color_continuous_scale='Viridis'
)

fig.show()

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 기본 groupby 집계 ⭐

**문제**: 다음 매출 데이터에서 지역별 총 매출을 계산하세요.

```python
data = pd.DataFrame({
    'region': ['서울', '부산', '서울', '대구', '부산'],
    'sales': [100, 150, 120, 90, 180]
})
```

**기대 결과**: 지역별 총 매출 Series

In [ ]:
data = pd.DataFrame({
    'region': ['서울', '부산', '서울', '대구', '부산'],
    'sales': [100, 150, 120, 90, 180]
})

# 여기에 코드를 작성하세요


---

### Q2. 다중 컬럼 groupby ⭐⭐

**문제**: 지역과 제품별 총 매출을 계산하고 DataFrame으로 변환하세요.

```python
data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산'],
    'product': ['노트북', '마우스', '노트북', '마우스'],
    'sales': [1200, 30, 1000, 25]
})
```

**기대 결과**: 지역, 제품, 총매출 컬럼을 가진 DataFrame

In [ ]:
data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산'],
    'product': ['노트북', '마우스', '노트북', '마우스'],
    'sales': [1200, 30, 1000, 25]
})

# 여기에 코드를 작성하세요


---

### Q3. agg() 복합 집계 ⭐⭐

**문제**: 제품별로 매출의 합계, 평균, 최대값을 동시에 계산하세요.

```python
data = pd.DataFrame({
    'product': ['노트북', '노트북', '마우스', '마우스'],
    'sales': [1200, 1500, 30, 25]
})
```

**기대 결과**: sum, mean, max 컬럼을 가진 DataFrame

In [ ]:
data = pd.DataFrame({
    'product': ['노트북', '노트북', '마우스', '마우스'],
    'sales': [1200, 1500, 30, 25]
})

# 여기에 코드를 작성하세요


---

### Q4. transform()으로 그룹 평균 추가 ⭐⭐⭐

**문제**: 지역별 평균 매출을 각 행에 추가한 후, 평균 대비 비율(%)을 계산하세요.

```python
data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산'],
    'sales': [100, 120, 150, 180]
})
```

**기대 결과**: region, sales, region_avg, ratio 컬럼을 가진 DataFrame

In [ ]:
data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산'],
    'sales': [100, 120, 150, 180]
})

# 여기에 코드를 작성하세요


---

### Q5. pivot_table 생성 ⭐⭐⭐

**문제**: 지역(행) x 제품(열) 형태의 피벗 테이블을 만들고, 결측치를 0으로 채우세요.

```python
data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '대구'],
    'product': ['노트북', '마우스', '노트북', '키보드'],
    'sales': [1200, 30, 1000, 80]
})
```

**기대 결과**: 지역이 인덱스, 제품이 컬럼인 피벗 테이블

In [ ]:
data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '대구'],
    'product': ['노트북', '마우스', '노트북', '키보드'],
    'sales': [1200, 30, 1000, 80]
})

# 여기에 코드를 작성하세요


---

### Q6. Inner Merge ⭐⭐⭐

**문제**: 주문 데이터와 제품 데이터를 product_id 기준으로 Inner Join하세요.

```python
orders = pd.DataFrame({
    'order_id': [1, 2, 3],
    'product_id': [101, 102, 103],
    'quantity': [2, 5, 1]
})

products = pd.DataFrame({
    'product_id': [101, 102, 104],
    'product_name': ['노트북', '마우스', '키보드'],
    'price': [1200, 30, 80]
})
```

**기대 결과**: product_id가 양쪽 모두 있는 행만 포함된 DataFrame

In [ ]:
orders = pd.DataFrame({
    'order_id': [1, 2, 3],
    'product_id': [101, 102, 103],
    'quantity': [2, 5, 1]
})

products = pd.DataFrame({
    'product_id': [101, 102, 104],
    'product_name': ['노트북', '마우스', '키보드'],
    'price': [1200, 30, 80]
})

# 여기에 코드를 작성하세요


---

### Q7. concat으로 데이터 결합 ⭐⭐⭐⭐

**문제**: 1월과 2월 매출 데이터를 위아래로 결합하고, 인덱스를 재설정하세요.

```python
jan_sales = pd.DataFrame({
    'product': ['노트북', '마우스'],
    'sales': [1200, 30]
})

feb_sales = pd.DataFrame({
    'product': ['노트북', '키보드'],
    'sales': [1350, 85]
})
```

**기대 결과**: 4행 2열 DataFrame (인덱스 0, 1, 2, 3)

In [ ]:
jan_sales = pd.DataFrame({
    'product': ['노트북', '마우스'],
    'sales': [1200, 30]
})

feb_sales = pd.DataFrame({
    'product': ['노트북', '키보드'],
    'sales': [1350, 85]
})

# 여기에 코드를 작성하세요


---

### Q8. Left Merge + 계산 ⭐⭐⭐⭐

**문제**: 주문 데이터에 제품 정보를 Left Join한 후, total_amount (수량 x 가격) 컬럼을 추가하세요.

```python
orders = pd.DataFrame({
    'order_id': [1, 2, 3],
    'product_id': [101, 102, 103],
    'quantity': [2, 5, 1]
})

products = pd.DataFrame({
    'product_id': [101, 102],
    'product_name': ['노트북', '마우스'],
    'price': [1200, 30]
})
```

**기대 결과**: order_id, product_id, quantity, product_name, price, total_amount 컬럼을 가진 DataFrame

In [ ]:
orders = pd.DataFrame({
    'order_id': [1, 2, 3],
    'product_id': [101, 102, 103],
    'quantity': [2, 5, 1]
})

products = pd.DataFrame({
    'product_id': [101, 102],
    'product_name': ['노트북', '마우스'],
    'price': [1200, 30]
})

# 여기에 코드를 작성하세요


---

### Q9. 다중 키 Merge ⭐⭐⭐⭐⭐

**문제**: 실제 매출과 목표 매출 데이터를 지역+제품 기준으로 조인하고, 달성률(%)을 계산하세요.

```python
actual = pd.DataFrame({
    'region': ['서울', '부산', '서울'],
    'product': ['노트북', '노트북', '마우스'],
    'sales': [1200, 1000, 30]
})

target = pd.DataFrame({
    'region': ['서울', '부산', '대구'],
    'product': ['노트북', '노트북', '노트북'],
    'target': [1500, 900, 800]
})
```

**기대 결과**: region, product, sales, target, achievement_rate(%) 컬럼을 가진 DataFrame

In [ ]:
actual = pd.DataFrame({
    'region': ['서울', '부산', '서울'],
    'product': ['노트북', '노트북', '마우스'],
    'sales': [1200, 1000, 30]
})

target = pd.DataFrame({
    'region': ['서울', '부산', '대구'],
    'product': ['노트북', '노트북', '노트북'],
    'target': [1500, 900, 800]
})

# 여기에 코드를 작성하세요


---

### Q10. 종합 문제: groupby + pivot + merge ⭐⭐⭐⭐⭐

**문제**: 다음 거래 데이터에서:
1. 지역별 총 매출을 계산하고
2. 목표 데이터와 조인한 후
3. 달성률을 계산하여
4. 지역을 인덱스로 하는 요약 테이블을 만드세요.

```python
transactions = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산', '대구'],
    'product': ['노트북', '마우스', '노트북', '키보드', '마우스'],
    'sales': [1200, 30, 1000, 80, 25]
})

targets = pd.DataFrame({
    'region': ['서울', '부산', '대구'],
    'target': [1500, 1200, 100]
})
```

**기대 결과**: region(인덱스), total_sales, target, achievement_rate 컬럼을 가진 DataFrame

In [ ]:
transactions = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산', '대구'],
    'product': ['노트북', '마우스', '노트북', '키보드', '마우스'],
    'sales': [1200, 30, 1000, 80, 25]
})

targets = pd.DataFrame({
    'region': ['서울', '부산', '대구'],
    'target': [1500, 1200, 100]
})

# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/문법 | 실무 활용 |
|------|----------------|----------|
| groupby 기본 | `df.groupby('컬럼')['값'].sum()` | 지역별, 제품별 집계 |
| 다중 컬럼 groupby | `df.groupby(['컬럼1', '컬럼2'])` | 세분화된 분석 (지역+제품) |
| agg() 복합 집계 | `df.groupby().agg(['sum', 'mean'])` | 월말 리포트 (합계+평균+최대) |
| transform() | `df.groupby().transform('mean')` | 그룹 평균 vs 개별 비교 |
| Plotly 시각화 | `px.bar()`, `px.line()` | 집계 결과 차트 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| pivot_table | `df.pivot_table(values, index, columns)` | 엑셀 피벗 테이블처럼 요약 |
| merge (inner) | `pd.merge(df1, df2, on='key')` | 양쪽 모두 있는 데이터만 |
| merge (left) | `pd.merge(df1, df2, how='left')` | 왼쪽 기준 VLOOKUP |
| concat (axis=0) | `pd.concat([df1, df2])` | 여러 파일 위아래 결합 |
| concat (axis=1) | `pd.concat([df1, df2], axis=1)` | 좌우 결합 |
| px.imshow() | `px.imshow(pivot_table)` | 피벗 테이블 히트맵 |

### 💡 실무 팁

1. **groupby 후 reset_index()**: Series를 DataFrame으로 변환하여 시각화 쉽게
2. **pivot_table의 fill_value=0**: 결측치를 0으로 채워 계산 오류 방지
3. **merge의 how 선택**: inner(교집합), left(왼쪽 기준), outer(합집합) 명확히 구분
4. **concat의 ignore_index=True**: 인덱스 자동 재설정으로 깔끔하게
5. **transform vs apply**: 원본 크기 유지하려면 transform, 새 값 계산하려면 apply
6. **히트맵 컬러맵**: Blues(파랑), RdYlGn(빨강-노랑-초록), Viridis(보라-노랑) 상황에 맞게 선택
7. **엑셀 VLOOKUP 대신 merge**: 속도 빠르고 여러 키로 조인 가능